In [1]:
# Importing relevant packages
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

In [2]:
# Cache data to improve dashboard performance
if 'data' not in pn.state.cache.keys():

    df = pd.read_csv('https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv')

    pn.state.cache['data'] = df.copy()

else: 

    df = pn.state.cache['data']
df

,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_cumulative_other_co2,share_global_flaring_co2,share_global_gas_co2,share_global_luc_co2,share_global_oil_co2,share_global_other_co2,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
0,Afghanistan,1850,AFG,3752993.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.121,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1851,AFG,3769828.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.118,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1852,AFG,3787706.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.116,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1853,AFG,3806634.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.115,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1854,AFG,3825655.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.114,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46518,Zimbabwe,2017,ZWE,14751101.0,2.194784e+10,0.469,0.032,9.596,-0.937,-8.899,...,NaN,0.0,0.0,0.219,0.026,NaN,115.59,28.30,0.910,9.486
46519,Zimbabwe,2018,ZWE,15052191.0,2.271535e+10,0.558,0.037,11.795,2.199,22.920,...,NaN,0.0,0.0,0.211,0.033,NaN,118.22,30.83,0.771,6.537
46520,Zimbabwe,2019,ZWE,15354606.0,NaN,0.570,0.037,11.115,-0.681,-5.772,...,NaN,0.0,0.0,0.183,0.030,NaN,117.96,30.53,0.978,8.795
46521,Zimbabwe,2020,ZWE,15669663.0,NaN,0.570,0.036,10.608,-0.507,-4.559,...,NaN,0.0,0.0,0.194,0.030,NaN,NaN,NaN,1.006,9.481


In [3]:
# Quick view of all the columns
df.columns

Index(['country', 'year', 'iso_code', 'population', 'gdp', 'cement_co2',
       'cement_co2_per_capita', 'co2', 'co2_growth_abs', 'co2_growth_prct',
       'co2_including_luc', 'co2_including_luc_growth_abs',
       'co2_including_luc_growth_prct', 'co2_including_luc_per_capita',
       'co2_including_luc_per_gdp', 'co2_including_luc_per_unit_energy',
       'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'coal_co2',
       'coal_co2_per_capita', 'consumption_co2', 'consumption_co2_per_capita',
       'consumption_co2_per_gdp', 'cumulative_cement_co2', 'cumulative_co2',
       'cumulative_co2_including_luc', 'cumulative_coal_co2',
       'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_luc_co2',
       'cumulative_oil_co2', 'cumulative_other_co2', 'energy_per_capita',
       'energy_per_gdp', 'flaring_co2', 'flaring_co2_per_capita', 'gas_co2',
       'gas_co2_per_capita', 'ghg_excluding_lucf_per_capita', 'ghg_per_capita',
       'land_use_change_co2', 'land_use_chang

## Data Processing

In [4]:
# Filling all empty cells with 0
df=df.fillna(0)

# Creating a new column of GDP per capita
df['gdp_per_capita']=np.where(df['population']!=0, df['gdp']/df['population'], 0)

In [5]:
# Creating an interactive dataframe
idf=df.interactive()

In [6]:
max_year = df['year'].max()
min_year = df['year'].min()

print('Years present in this dataset range from ' + str(min_year) + ' to ' + str(max_year))

Years present in this dataset range from 1750 to 2021


In [7]:
# Year slider widget
year_slider = pn.widgets.IntSlider(name='Year', start=1750, end=2021, step=1, value=1951, bar_color='green')
year_slider

IntSlider(bar_color='green', end=2021, name='Year', start=1750, value=1951)

## CO2 Emission Over Time by Continent

In [8]:
# CO2 and CO2 per capita widgets
yaxis_co2 = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['co2', 'co2_per_capita',],
    button_type='success'
)

In [9]:
continents = ['World', 'Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antartica']

# Define data pipeline
co2_pipeline = (
    idf[
        (idf.year <= year_slider) &
        (idf.country.isin(continents))
    ]
    .groupby(['country', 'year'])[yaxis_co2].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year', ascending=False)
    .reset_index(drop=True)
)

In [10]:
co2_table = co2_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width') 
co2_table

In [11]:
co2_plot = co2_pipeline.hvplot(x = 'year', by='country', y=yaxis_co2,line_width=2, title="CO\N{SUBSCRIPT TWO} Emission by Continent")
co2_plot

## Chnage in CO2 Over Time by Continent

In [12]:
co2_growth_abs_pipeline = (
    idf[
    (idf.year <= year_slider) &
    (idf.country.isin(continents))
]
    .groupby(['country', 'year'])['co2_growth_abs'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year', ascending=False)
    .reset_index(drop=True)
)                           

In [13]:
co2_growth_plot = co2_growth_abs_pipeline.hvplot(x = 'year', by='country', y='co2_growth_abs',line_width=2, title="CO\N{SUBSCRIPT TWO} Growth by Continent")
co2_growth_plot

## CO2 vs GDP per Capita Scatter Plot

In [14]:
countries_to_exempt = ['World', 'Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antartica', 'Africa (GCP)','Asia (GCP)','Asia (excl. China and India)','Central America (GCP)','Europe (GCP)','Europe (excl. EU-27)','Europe (excl. EU-28)','European Union (27)','European Union (27) (GCP)','European Union (28)','French Equatorial Africa (GCP)','French West Africa (GCP)','High-income countries','International transport','Kuwaiti Oil Fires (GCP)','Leeward Islands (GCP)','Low-income countries','Lower-middle-income countries','Micronesia (country)','Middle East (GCP)','Non-OECD (GCP)','North America (GCP)','North America (excl. USA)','OECD (GCP)','Oceania (GCP)','Panama Canal Zone (GCP)','Ryukyu Islands (GCP)','Saint Martin (French part)','Sint Maarten (Dutch part)','South America (GCP)','St. Kitts-Nevis-Anguilla (GCP)','Upper-middle-income countries']

co2_vs_gdp_pipeline = (
    idf[
        (idf.year == year_slider) &
        (~ (idf.country.isin(countries_to_exempt)))
    ]
    .groupby(['country', 'year', 'gdp_per_capita'])['co2'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='co2', ascending=False)  
    .reset_index(drop=True)
)

In [15]:
co2_vs_gdp_scatterplot = co2_vs_gdp_pipeline.hvplot(x='gdp_per_capita', 
                                                    y='co2', 
                                                    by='country', 
                                                    size=80, kind="scatter", 
                                                    alpha=0.7,
                                                    legend=False, 
                                                    height=500, 
                                                    width=500)
co2_vs_gdp_scatterplot

## CO2 Source by Continent

In [16]:
yaxis_co2_source = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['coal_co2', 'oil_co2', 'gas_co2'], 
    button_type='success'
)

continents_excl_world = ['Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

co2_source_bar_pipeline = (
    idf[
        (idf.year == year_slider) &
        (idf.country.isin(continents_excl_world))
    ]
    .groupby(['year', 'country'])[yaxis_co2_source].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

In [17]:
co2_source_bar = co2_source_bar_pipeline.hvplot(kind='bar', 
                                                     x='country', 
                                                     y=yaxis_co2_source, 
                                                     title='CO\N{SUBSCRIPT TWO} source by continent',
                                                    color='#88d8b0')
co2_source_bar

## Creating and Launching Dashboard

In [18]:
# Creating the dashboard
template = pn.template.FastListTemplate(
    title='Carbon Dioxide Emmissions from the World', 
    sidebar=[pn.pane.Markdown("# CO\N{SUBSCRIPT TWO} Emissions and Climate Change"), 
             pn.pane.Markdown("#### Author: Sahil Kunder"),
             pn.pane.Markdown("#### The latest data from the world shows that carbon dioxide emissions will reach to 33.8 billion tonnes by 2022, an increase of almost 300 million tonnes. A significantly smaller rise when compared to the 2 billion tonnes increase in 2021, recovering from the economic crisis triggered by the pandemic. Whereas, this year majority of the emissions are caused by power generation and air travel as the world tries to get back on its feet."), 
             pn.pane.PNG('world.png', sizing_mode='scale_both'),
             pn.pane.Markdown("## Year Slider"),   
             year_slider],
    main=[pn.Row(pn.Column(yaxis_co2, 
                           co2_plot.panel(width=700), margin=(0,25)), 
                 co2_table.panel(width=500)), 
          pn.Row(pn.Column(co2_vs_gdp_scatterplot.panel(width=600), margin=(0,25)), 
                 pn.Column(yaxis_co2_source, co2_source_bar.panel(width=600))),
          pn.Row(pn.Column(co2_growth_plot.panel(width=1200)))
         ],
        
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable();